In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A3_017, input_features_A3_017 = pre_models.Input_A3_017('weights/')
    
    outputs = ['Input_A3_017']

    preprocessings = ['min_max_normalization']

    models = [model_A3_017]

    input_features = [input_features_A3_017]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=5
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_017.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A1_016', 'Input_A4_016', 'Input_A2_019',
       'Input_A1_018'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.009011243794593901
[0.010197434089272806, 0.00848800223464646, 0.01002208325952512, 0.008799768658811971, 0.00997705106380258, 0.008371288122635746, 0.009196408361391318, 0.009104648046390799, 0.008256042752416174, 0.007699711357046031]


test_r2_ave: 0.5276418274062353
[0.46913439973610815, 0.5894040313690869, 0.509533241261001, 0.5721071781060474, 0.47389286224087823, 0.47341608101953925, 0.49465805063476687, 0.45503829465946966, 0.6215946433915412, 0.6176394916439145]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,5)

train_rmse: 0.009021544601542492
test_rmse: 0.009185245312196517
test_r2: 0.5194517040383431


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,3e-07)

test_rmse_ave: 0.00915265946598786
[0.01062750198763127, 0.008673317787159884, 0.00994386176369197, 0.008939578064045368, 0.009945566589734396, 0.008430424097449754, 0.009745321367017846, 0.009006018749580881, 0.008403021442409763, 0.007811982811157472]


test_r2_ave: 0.5127135025130259
[0.42341258348195965, 0.5712795215682764, 0.5171594651920497, 0.5584025737690227, 0.4772080844813562, 0.4659500753571916, 0.4325322480300323, 0.46678132108470427, 0.6080015474223772, 0.6064076047432891]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.04],0.01)

test_rmse_ave: 0.009230097048787068
[0.010712850925479455, 0.008792130351253567, 0.010074765589108081, 0.009202803075028248, 0.009927527433509496, 0.008270915189682867, 0.009711517230252325, 0.009112071378092598, 0.008554092848794324, 0.007942296466669701]


test_r2_ave: 0.5052574206935073
[0.41411430630033175, 0.5594533054152979, 0.504363289356982, 0.532014120812297, 0.4791028327153405, 0.48596800935175777, 0.43646223384147165, 0.4541492804609195, 0.5937799746249461, 0.5931668540557293]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.009154731992021709
test_rmse: 0.009323699506015254
test_r2: 0.5042389568202571


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.4,
              reg_alpha =  0.05,
              reg_lambda = 2)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.009313335268602781
[0.010598929677171258, 0.00884100326981116, 0.010250998065664861, 0.009141604843163866, 0.010236671587176564, 0.008094949510017158, 0.010105390019861934, 0.009481101702687733, 0.008478444859080529, 0.007904259151392748]


test_r2_ave: 0.4956756506473011
[0.42650875579453085, 0.5545419486391667, 0.48687181646100275, 0.5382175962211698, 0.44615614063095754, 0.5076076448240685, 0.38982414861165493, 0.4090411084039933, 0.600933006489341, 0.5970543403971253]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 8,
            min_samples_leaf =3,
            random_state = 42)
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.00926927863567096
[0.009960376687364077, 0.009191904829756122, 0.01024475199640912, 0.008475154918851924, 0.010505539986227328, 0.00874404108960199, 0.00997960197902898, 0.00966823117286096, 0.008206411067431944, 0.007716772629177148]


test_r2_ave: 0.4977234188693435
[0.49352933414788924, 0.5184795359647612, 0.48749693758891477, 0.6030938921449063, 0.4166804068772473, 0.4254770359226312, 0.40492007856865697, 0.38548326679016476, 0.6261305802295773, 0.6159431204586858]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=3e-07)
elastic_mod=ElasticNet(alpha=[0.04], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.4,
              reg_alpha =  0.05,
              reg_lambda = 2)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 8,
            min_samples_leaf =3,
            random_state = 42)
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.009111587199217902
[0.010493535849133723, 0.00863080060455602, 0.009958997163004192, 0.008846289046533617, 0.00993520073524237, 0.00833247936643075, 0.009716151348488971, 0.00915483693264715, 0.00827206866735412, 0.007775512278788097]


test_r2_ave: 0.5168309992130968
[0.43785743440725167, 0.5754724520960577, 0.515688498246291, 0.567571068791503, 0.4782972855400115, 0.4782871824522027, 0.43592429035601354, 0.44901359026117327, 0.620124162198968, 0.6100740277814962]


In [10]:

stack_mod = StackingRegressor(regressors=[lasso_mod,elastic_mod,lr, random_mod ], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.009163690737815314
[0.01015018384836756, 0.008951775586143596, 0.009969671197119515, 0.008517597416183683, 0.010116073835240458, 0.008756458258043652, 0.009843349740805963, 0.009413360396001139, 0.00802227946802538, 0.007896157632222199]


test_r2_ave: 0.5093197188947043
[0.4740425784976353, 0.5433093891132332, 0.5146497736099804, 0.5991086276601254, 0.459128889666503, 0.42384414931887227, 0.4210584915355349, 0.41745559771864404, 0.6427197714596238, 0.5978799203668911]


In [11]:
ave(skew_data[title],y,0.3,stack_mod,0.6,vote_mod,0.1,lasso_mod)

test_rmse_ave: 0.009097463605925865
[0.010389034661647686, 0.008693364570252635, 0.009922861223718637, 0.008726271689202548, 0.00995854859461722, 0.008429047099369355, 0.009724686932684141, 0.009184902647150935, 0.008171895639348838, 0.007774023001266643]


test_r2_ave: 0.5178495732701288
[0.4489980182741087, 0.5692954137756134, 0.5191967430429572, 0.5792249772934348, 0.47584238718899896, 0.46612452106829994, 0.434932780366171, 0.44538862077905916, 0.6292688890173914, 0.6102233818952523]


In [46]:
name='Input_A3_017'
test_pred=[]
seed=[9,8,5,3,1,7]
stack_w=0.3
vote_w=0.6
model_w=0.1
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)





In [47]:
nn_test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑

In [48]:
nn_pred = predict_df(nn_test)
test_pred.append(nn_pred.to_numpy().reshape(1,95))


In [49]:
final_test = pd.DataFrame(np.mean(np.array(test_pred), axis=0).reshape(-1, 1), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))

In [50]:
final_test

,Input_A3_017
0,-0.037785
1,-0.026253
2,-0.044509
3,-0.045688
4,-0.018948
...,...
90,-0.015150
91,-0.021981
92,-0.035962
93,-0.022403
